In [1]:
LLAMA_STACK_URL = "http://localhost:8321/"
MCP_URL = "http://127.0.0.1:8000/mcp"
LLAMA_STACK_MODEL_IDS = [
    "openai/gpt-3.5-turbo",
    "openai/gpt-4o",
    "llama-openai-compat/Llama-3.3-70B-Instruct",
    "watsonx-Llama-3.3-70B-Instruct"
]

# Using gpt-4o for this demo, but feel free to try one of the others or add more to run.yaml.
LLAMA_STACK_MODEL_ID = LLAMA_STACK_MODEL_IDS[1]

In [2]:
from llama_stack_client import LlamaStackClient

client = LlamaStackClient(base_url="http://localhost:8321")

In [4]:
client.chat.completions.create(
    model=LLAMA_STACK_MODEL_ID,
    messages=[{"role": "user", "content": "What is the capital of France?"}]
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/chat/completions "HTTP/1.1 200 OK"


OpenAIChatCompletion(id='chatcmpl-C8TyjyCgldvbvWTJm7aDRiBxJbmMG', choices=[OpenAIChatCompletionChoice(finish_reason='stop', index=0, message=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content='The capital of France is Paris.', name=None, tool_calls=None, refusal=None, annotations=[], audio=None, function_call=None), logprobs=None)], created=1756137897, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_46bff0e0c8', usage={'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}})

In [10]:
def print_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output items: {len(response.output)}")
    
    for i, output_item in enumerate(response.output):
        if len(response.output) > 1:
            print(f"\n--- Output Item {i+1} ---")
        print(f"Output type: {output_item.type}")
        
        if output_item.type in ("text", "message"):
            print(f"Response content: {output_item.content[0].text}")
        elif output_item.type == "file_search_call":
            print(f"  Tool Call ID: {output_item.id}")
            print(f"  Tool Status: {output_item.status}")
            # 'queries' is a list, so we join it for clean printing
            print(f"  Queries: {', '.join(output_item.queries)}")
            # Display results if they exist, otherwise note they are empty
            print(f"  Results: {output_item.results if output_item.results else 'None'}")
        elif output_item.type == "mcp_list_tools":
            print_mcp_list_tools(output_item)
        elif output_item.type == "mcp_call":
            print_mcp_call(output_item)
        else:
            print(f"Response content: {output_item.content}")
def print_mcp_call(mcp_call):
    """Print MCP call in a nicely formatted way"""
    print(f"\n🛠️  MCP Tool Call: {mcp_call.name}")
    print(f"   Server: {mcp_call.server_label}")
    print(f"   ID: {mcp_call.id}")
    print(f"   Arguments: {mcp_call.arguments}")
    
    if mcp_call.error:
        print("Error: {mcp_call.error}")
    elif mcp_call.output:
        print("Output:")
        # Try to format JSON output nicely
        try:
            import json
            parsed_output = json.loads(mcp_call.output)
            print(json.dumps(parsed_output, indent=4))
        except:
            # If not valid JSON, print as-is
            print(f"   {mcp_call.output}")
    else:
        print("   ⏳ No output yet")

def print_mcp_list_tools(mcp_list_tools):
    """Print MCP list tools in a nicely formatted way"""
    print(f"\n🔧 MCP Server: {mcp_list_tools.server_label}")
    print(f"   ID: {mcp_list_tools.id}")
    print(f"   Available Tools: {len(mcp_list_tools.tools)}")
    print("=" * 80)
    
    for i, tool in enumerate(mcp_list_tools.tools, 1):
        print(f"\n{i}. {tool.name}")
        print(f"   Description: {tool.description}")
        
        # Parse and display input schema
        schema = tool.input_schema
        if schema and 'properties' in schema:
            properties = schema['properties']
            required = schema.get('required', [])
            
            print("   Parameters:")
            for param_name, param_info in properties.items():
                param_type = param_info.get('type', 'unknown')
                param_desc = param_info.get('description', 'No description')
                required_marker = " (required)" if param_name in required else " (optional)"
                print(f"     • {param_name} ({param_type}){required_marker}")
                if param_desc:
                    print(f"       {param_desc}")
        
        if i < len(mcp_list_tools.tools):
            print("-" * 40)

In [12]:
mcp_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="List all the products available in the ElectroShop",
    tools=[
        {
            "type": "mcp",
            "server_url": MCP_URL,
            "server_label": "ElectroShop Sales MCP",
        }
    ]
)

print_response(mcp_llama_stack_client_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ID: resp-088e3035-ade3-408a-8bd6-1c198038b7f5
Status: completed
Model: openai/gpt-4o
Created at: 1756138166
Output items: 3

--- Output Item 1 ---
Output type: mcp_list_tools

🔧 MCP Server: ElectroShop Sales MCP
   ID: mcp_list_232711ba-6107-492f-a35c-c89483012023
   Available Tools: 4

1. get_customers
   Description: Returns list of registered customers
    
   Parameters:
----------------------------------------

2. insert_customer_into_db
   Description: Add new customer to database
Args:
    customer_data:
        first_name: Customer's first name
        last_name: Customer's last name
        address: Customer's address
        city: Customer's city
        country: Customer's country
        phone_number: Customer's phone number
        email: Customer's email address

Returns:
    Dictionary containing the newly created customer data including customer_id
   Parameters:
     • customer_data (object) (required)
----------------------------------------

3. place_order
   Descrip